# ПОСТАНОВКА ЗАДАЧИ
Довольно часто приходится добывать информацию не с помощью удобных API, а прямо с HTML-страниц. Получить содержимое страницы в большинстве случаев несложно, труднее извлечь из HTML-кода нужную информацию. В качестве мы рассмотрим страницу новости на Lenta.ru, из которой будем доставать полезную информацию:

1. Заголовок страницы

2. Редакторский заголовок перед текстом

3. Дату публикации

4. Текст публикации

5. Все ссылки на странице

# ПОЛУЧАЕМ СОДЕРЖИМОЕ СТРАНИЦЫ
Сначала сделаем веб-запрос к странице новости и получим ее HTML-код:

In [1]:
import requests

url = 'https://lenta.ru/articles/2018/07/01/pobeda/'

r = requests.get(url)

In [2]:
assert r.status_code == 200

In [4]:
print(r.text)

<!DOCTYPE html><html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"><head><title>Россия уделала Испанию на чемпионате мира. Вся страна в шоке и ликует: Coцсети: Интернет и СМИ: Lenta.ru</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type" />
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"66a8d51230","applicationID":"1241738","transactionName":"J19cQUoOWA0ERBoQXhRZUUYXElwOFg==","queueTime":0,"applicationTime":131,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQUGU1VRGwICUFBVBAk="};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e

# БИБЛИОТЕКА BEAUTIFULSOUP
Итак, содержимое страницы у нас лежит в r.text. Для того, чтобы извлекать нужную нам информацию воспользуемся библиотекой BeautifulSoup. Переведем содержимое страницы в объект BeautifulSoup:

In [5]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, 'html.parser')

In [6]:
print(soup)

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"><head><title>Россия уделала Испанию на чемпионате мира. Вся страна в шоке и ликует: Coцсети: Интернет и СМИ: Lenta.ru</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"66a8d51230","applicationID":"1241738","transactionName":"J19cQUoOWA0ERBoQXhRZUUYXElwOFg==","queueTime":0,"applicationTime":131,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQUGU1VRGwICUFBVBAk="};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e

Однако, теперь мы можем получать нужные нам элементы страницы, обращаясь к soup. Например, для получения заголовка используем метод title:

In [7]:
soup.title

<title>Россия уделала Испанию на чемпионате мира. Вся страна в шоке и ликует: Coцсети: Интернет и СМИ: Lenta.ru</title>

In [8]:
soup.title.text

'Россия уделала Испанию на чемпионате мира. Вся\xa0страна в шоке и ликует: Coцсети: Интернет и СМИ: Lenta.ru'

# РЕДАКТОРСКИЙ ЗАГОЛОВОК
Заголовок из тэга <title\> это довольно просто. Но как мы видим на странице реальный заголовок, который видят пользователи, отличается:

Нам потребуется ID этого заголовка на странице. Откройте страницу новости, наведите курсор на заголовок и через правый клик мыши (или нажатие двумя пальцами на тачпаде) откройте меню. Найдите пункт "Исследовать элемент" и откройте его. В Google Chrome и Яндекс.Браузере это будет выглядеть так:

Все, что нам потребуется - это название класса class_='b-topic__title' в коде этого элемента. Просто задаем библиотеке BeautifulSoup найти этот элемент:

In [9]:
soup.find(class_='b-topic__title')

<h1 class="b-topic__title" itemprop="headline">«Простите, парни, что поливал вас дерьмом на каждом углу»</h1>

In [10]:
soup.find(class_='b-topic__title').text

'«Простите, парни, что поливал вас дерьмом на\xa0каждом углу»'

# ДАТА ПУБЛИКАЦИИ
Давайте вытащим дату публикации со страницы новости:

Наводим на дату и нажимаем 'Исследовать элемент'. Можно искать по названию класса 'g-date'. Но если на странице несколько элементов с таким названием, то мы можем получить не то что хотим. Давайте сначала возьмем содержимое элемента class_='b-topic__info' (на скриншоте этот элемент чуть выше выделенной синим строчки на один уровень вложенности выше).

In [14]:
topic = soup.find('div', class_='b-topic__info')
topic

<div class="b-topic__info"><time class="g-date" datetime="2018-07-01T18:20:00Z" itemprop="datePublished" pubdate=""> 21:20,  1 июля 2018</time><h1 class="b-topic__title" itemprop="headline">«Простите, парни, что поливал вас дерьмом на каждом углу»</h1><h2 class="b-topic__rightcol">Россия уделала Испанию на чемпионате мира. Вся страна в шоке и ликует</h2><div class="b-topic__socials _header"><div class="b-socials"><div class="b-socials__sharing js-sharing"></div><div class="b-socials__favorite"><div class="b-favorite js-favorite" data-class-active="b-favorite_active" data-class-loaded="b-favorite_loaded" data-class-logged="b-favorite_logged" data-uri="/articles/2018/07/01/pobeda/"><a class="b-favorite__item b-favorite__button js-favorite__button" href="/auth/signin/"><svg class="b-favorite__button-icon svg-icons__ui-shevron b-favorite__icon-shevron"><use xlink:href="#ui-chevron"></use></svg><svg class="b-favorite__button-icon svg-icons__ui-shevron_stroke b-favorite__icon-shevron_stroke"

Нужная нам дата где-то внутри переменной topic. Зададим теперь поиск внутри topic и получим дату публикации:

In [15]:
topic.find(class_ = 'g-date').text

' 21:20,  1 июля 2018'

# ТЕКСТ НОВОСТИ
Искать можно не только по классам, но и по другим атрибутам элементов. Попробуйте самостоятельно найти в коде страницы атрибуты основного текста новости:
<div class="b-text clearfix js-topic__text" itemprop="articleBody"\>

In [16]:
soup.find( itemprop = 'articleBody' ).text

'Сборная России обыграла Испанию и вышла в четвертьфинал чемпионата мира по футболу. Многие ждали от встречи с сильнейшей команды планеты хотя бы нестыдной борьбы, а в итоге получили исторический матч — 4:3, результат превзошел все ожидания. Шок, ликование на улицах, а в соцсетях\xa0— пожар, эмоции переполняют болельщиков.Пользователи сети благодарят команду и извиняются перед тренером, в которого еще совсем недавно не верили.\n—\n  милькøш 击 (@lixoradkaenota)\n  01 июля 2018, 16:54\n\n\n\n—\n  созерцатель (@newfag_in_life)\n  01 июля 2018, 16:42\n\n\n\n—\n  Vitaly Petrov (@vitalypetrov)\n  01 июля 2018, 17:02\n\n\n\n—\n  Борисыч (@Borisich_glass)\n  01 июля 2018, 16:54\n\n\n\n—\n  Anton Ziuzenok (@a_ziuzenok)\n  01 июля 2018, 17:16\n\n\n\n—\n  Станислав Рынкевич (@salt1s)\n  01 июля 2018, 16:51\n\n\n\n—\n  Марат Касем (@MaratQasem)\n  01 июля 2018, 17:12\n\n\n\n—\n  Ромыч (@Instigator_88)\n  01 июля 2018, 17:03\n\n\n\n—\n  Milligan (@milligantop)\n  01 июля 2018, 17:03\n\n\nЧемпионат 

# ССЫЛКИ В ТЕКСТЕ НОВОСТИ
Отдельно рассмотрим поиск ссылок на другие страницы в тексте новости. В самом простом случае ссылка на HTML-странице задается тэгом 'a', у которого в параметре 'href' прописывается текст ссылки:

<a href="https://some_site.ru"\>

Зададим поиск по тэгу 'a':

In [20]:
for link in soup.find(itemprop='articleBody').find_all('a'):
    print(link)

<a href="https://lenta.ru/news/2018/07/01/russia_win/" target="_blank">обыграла</a>
<a href="https://twitter.com/lixoradkaenota/status/1013465983061778433"><time datetime="2018-07-01T16:54:54+00:00">01 июля 2018, 16:54</time></a>
<a href="https://twitter.com/newfag_in_life/status/1013462966782582785"><time datetime="2018-07-01T16:42:55+00:00">01 июля 2018, 16:42</time></a>
<a href="https://twitter.com/vitalypetrov/status/1013467931257311232"><time datetime="2018-07-01T17:02:38+00:00">01 июля 2018, 17:02</time></a>
<a href="https://twitter.com/Borisich_glass/status/1013465759799042048"><time datetime="2018-07-01T16:54:01+00:00">01 июля 2018, 16:54</time></a>
<a href="https://twitter.com/a_ziuzenok/status/1013471409979445249"><time datetime="2018-07-01T17:16:28+00:00">01 июля 2018, 17:16</time></a>
<a href="https://twitter.com/salt1s/status/1013465249989775360"><time datetime="2018-07-01T16:51:59+00:00">01 июля 2018, 16:51</time></a>
<a href="https://twitter.com/MaratQasem/status/1013470

И теперь просто извлечем значение href из этих ссылок:

In [21]:
for link in soup.find( itemprop = 'articleBody' ).find_all( 'a' ):
    print( link.get('href') )

https://lenta.ru/news/2018/07/01/russia_win/
https://twitter.com/lixoradkaenota/status/1013465983061778433
https://twitter.com/newfag_in_life/status/1013462966782582785
https://twitter.com/vitalypetrov/status/1013467931257311232
https://twitter.com/Borisich_glass/status/1013465759799042048
https://twitter.com/a_ziuzenok/status/1013471409979445249
https://twitter.com/salt1s/status/1013465249989775360
https://twitter.com/MaratQasem/status/1013470315492585474
https://twitter.com/Instigator_88/status/1013468227370962944
https://twitter.com/milligantop/status/1013468156373950464
https://twitter.com/she_lova_/status/1013463135477551108
https://twitter.com/Fall_Out_Daria/status/1013463774253248512
/tags/persons/akinfeev-igor/
https://twitter.com/valyagorushkina/status/1013467293144305665
https://twitter.com/Alisa06101/status/1013466591034511369
https://twitter.com/quad_axels/status/1013469038406701056
https://twitter.com/GuyFloral/status/1013467692601184256
https://twitter.com/maleenkiymyc/st